# Project 3.1.1 Data Cleaning and Data Science Process
---
Following scraping textual information from reddit posts, we now continue with the data science process of data cleaning, EDA, model building, evaluation and lastly recommendation of business solutions.

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
# increase default figure and font sizes for easier viewing
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

# always be stylish
plt.style.use('ggplot')

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.metrics import confusion_matrix

In [ ]:
# Read in google home reddit data
df_gh = pd.read_csv('./reddit_gh.csv')
df_gh.head()

The columns with the more relevant textual data are from subreddit, selftext and title. 

In [ ]:
# Set a new dataframe with the columns we want ()
df_gh1 = pd.DataFrame(df_gh[['subreddit', 'selftext', 'title']])
# Shape of dataframe
df_gh1.shape

In [ ]:
# Drop duplicates in selftext column
df_gh1.drop_duplicates(subset='selftext',inplace=True)
# Shape of dataframe after removing duplicates
df_gh1.shape

In [ ]:
# Drop NAs.
df_gh1.dropna(inplace=True)
# Shape of dataframe after removing NAs
df_gh1.shape

In [ ]:
# Review first 5 rows of dataframe
df_gh1.head()

In [ ]:
# Read in google pixel reddit data
df_gp = pd.read_csv('./reddit_gp.csv')
df_gp.head()

In [ ]:
# Set a new dataframe with the columns we want ()
df_gp1 = pd.DataFrame(df_gp[['subreddit', 'selftext', 'title']])
# Shape of dataframe
df_gp1.shape

In [ ]:
# Drop duplicates in selftext column
df_gp1.drop_duplicates(subset='selftext',inplace=True)
# Shape of dataframe after removing duplicates
df_gp1.shape

In [ ]:
# Drop NAs.
df_gp1.dropna(inplace=True)
# Shape of dataframe after removing NAs
df_gp1.shape

In [ ]:
# Join the two dataframes together
df = pd.concat([df_gh1,df_gp1])
df.reset_index(drop=True, inplace=True)
# Shape of concatenated dataframe 
df.shape

In [ ]:
df['subreddit'].value_counts()

In [ ]:
# Engineer a feature to turn subreddit into a 1/0 column, where 1 indicates googlehome.
df['googlehome'] = df['subreddit'].map({'googlehome': 1, 'GooglePixel': 0})
df.head(2)

In [ ]:
df['googlehome'].value_counts(normalize=True)

The distribution of the classes is quite balanced. If the class is imblanced, the resulting model is likely to have low predictive accuracy for the infrequent class.

### Basic EDA
---
Building good performing classifers from data with easily separable classes is relatively straightforward but care should be excercised to determine if such data is representative of the actual world environment. In most real world environment, samples from different classes share similar characteristics or are overlapped. This means the boundaries of each class may not be clearly defined as desired. 

Common practices to address this problem includes:
- i) modifying the original data by introducing/removing features which decrease the overlapping region,
- ii) adapting algorithms to reduce the negative impact of overlapping features.

Through the basic EDA process, we set out to discover what are the prevalent key words among the reddit posts, and their distribution across text data of googlehome and googlepixel reddit posts. With these insights we would then be in better position to fine-tune the subsequent models.

In [ ]:
text = df['selftext']
text[0:5]

In [ ]:
'''
create a document term matrix with:
-ngram range 1,2
-minimum document appearance for any term = 2
-removal of all English stop words
'''
# Instantiate countvectorizer
cvec = CountVectorizer(ngram_range=(1, 2), stop_words='english', min_df=2)
matrix = cvec.fit_transform(text)

In [ ]:
# Create a dataframe with feature names = words
tf = pd.DataFrame(cvec.fit_transform(text).toarray(), columns=cvec.get_feature_names())

In [ ]:
# see most common terms
tf.sum().sort_values(ascending=False).head(10)

In [ ]:
# Create googlehome text df
googlehome_tdf = df[df['googlehome'] ==1]
googlehome_tdf.shape
googlehome_tdf.head()

In [ ]:
# Create googlepixel text df
googlepixel_tdf = df[df['googlehome'] == 0]
googlepixel_tdf.shape

In [ ]:
# vectorize selftext from googlehome text
tf1 = pd.DataFrame(cvec.fit_transform(googlehome_tdf['selftext']).toarray(), columns=cvec.get_feature_names())

In [ ]:
# No.of features fitted
tf1.shape

In [ ]:
# check value counts
tf1.sum().sort_values(ascending=False).head(10)

As expected, the words `google`, `home` and `google home` have high prevalence in google home reddit posts.

In [ ]:
# vectorize selftext from googlehome text
tf2 = pd.DataFrame(cvec.fit_transform(googlepixel_tdf['selftext']).toarray(), columns=cvec.get_feature_names())

In [ ]:
# No.of features fitted
tf2.shape

In [ ]:
# check value counts
tf2.sum().sort_values(ascending=False).head(10)

The words `google`, `phone` and `pixel` have high prevalence in google pixel reddit posts. Are any of these words common among the two reddit groups, and what is their distribution? Let's find out.

In [ ]:
# word counts of googlehome text and google pixel text
"""Merge the two dataframes on Outer join, align with both dataframe indexes."""
word_counts = pd.merge(pd.DataFrame(data=tf1.sum().sort_values(ascending=False)),\
                       pd.DataFrame(data=tf2.sum().sort_values(ascending=False)),\
                       how='outer',left_index=True,right_index=True)
print(word_counts)

In [ ]:
# rename columns
word_counts.columns = ['googlehome', 'googlepixel']

In [ ]:
# view the df to ensure correct column naming
word_counts.head(2)

In [ ]:
# find sums
word_counts['sum'] = word_counts['googlehome'] + word_counts['googlepixel']
# sort by most used values
word_counts.sort_values(['sum'], ascending=False).head(10)

In [ ]:
# Minimum occurrence of words
word_counts.sort_values(['sum'], ascending=True).head(5)

In [ ]:
# plot 20 most used words
word_counts.sort_values(['googlehome','googlepixel'], ascending=False).drop('sum', axis=1).head(30).\
plot(kind='bar',figsize=(15,6),fontsize=12,title='Top 20 common words across reddits');

In [ ]:
print(f"googlehome total posts: {df_gh1.shape[0]}, word count: {word_counts['googlehome'].sum()}")
print(f"googlepixel total posts: {df_gp1.shape[0]}, word count: {word_counts['googlepixel'].sum()}")

While the word `google` is unexpectedly common with 1731 instances, an interestingly observation is that it is not as prevalent for google pixel posts compared to google home. This could be due to consumers referring the google handphone simply as `pixel`. 
For (binary) classification problem, the ideal case is perfect separation in features for the classes. However, this is rarely the case in actual world environment. Observe that there are words that are common between `googlehome` and `googlepixel`. 
We remove words that have two characteristics:
- significant overlaps (smallest difference in word count occurrence in both classes)
- occur frequently. From the plot above, word count occurrence of more than 200 is determined emohirically to be a suitable  measure. 

In [ ]:
# find words with 
# 1. significant overlaps
# 2. measure by gap (difference of word count between googlehome & googlepixel class)
# 3. identify the more significant occuring ones by population proportion
word_counts['gap'] = abs(word_counts['googlehome'] - word_counts['googlepixel'])
# sort by most used values
word_counts.sort_values(['gap'], ascending=True).loc[(word_counts['sum']>300) & (word_counts['gap']<50)]

In [ ]:
word_counts.sort_values(['gap'], ascending=True).loc[(word_counts['sum']>200) & (word_counts['gap']<50)]

The top 10 words commonly overlapping words are [`does`, `work`, `time`, `want`, `use`, `using`, `way`, `assistant`, `ve`, `tried`] are the more common words among the two reddits with significant overlap (similar distribution, with word count aprrox. 200). To measure the impact on model predictive accuracy, it is proposed compare a logreg model without removing these key words and another logreg model with these key words removed.  

### Word Art
---
Following the EDA process, we received a request from the marketing department. They wanted to understand the hot topics for google home and google pixel by inference of the keywords from Reddit posts. While word cloud is technically not data science, we make a small detour to help them out.

In [ ]:
# grab libraries
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from matplotlib.pyplot import imread
%matplotlib inline

import numpy as np

In [ ]:
# create a list of all the text from googlehome
wa_gh = pd.DataFrame(cvec.fit_transform(googlehome_tdf['selftext']).toarray(), columns=cvec.get_feature_names())
all_textgh = []
for x in wa_gh:
    all_textgh.append(x)

In [ ]:
# create a list of all the text from googlepixel
wa_gp = pd.DataFrame(cvec.fit_transform(googlepixel_tdf['selftext']).toarray(), columns=cvec.get_feature_names())
all_textgp = []
for x in wa_gp:
    all_textgp.append(x)

In [ ]:
# format it into str
# In Python 3, unicode has been renamed to str.
googlehome_text = str(all_textgh)
googlepixel_text = str(all_textgp)

In [ ]:
# Define function to generate wordart (wordcloud)
def cloud(source_text):
    stop_words = ["ve'"] + list(STOPWORDS)
    wordcloud = WordCloud(max_words=200,\
                          stopwords=stop_words,\
                          background_color="black",\
                          min_font_size=10,\
                          colormap='viridis').generate(source_text)
    
    plt.figure(figsize=(9,6))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
# Generate word cloud image for googlehome
cloud(googlehome_text)

In [ ]:
# Generate word cloud image for googlepixel
cloud(googlepixel_text)

### Model Prep
---
First split data into train and test set, then fit only on the train set and transform the test set. This is to prevent data leakage from the test set to the train set (i.e. resultant model will be overfitted and not generalize well to new, unseen data). In other words, excercise caution not to inadvertently countvectorize (a transformer) the data first before doing train-test-split.

In [ ]:
# Create train test split
# X is selftext. y is googlehome.
X_train,X_test,y_train,y_test = train_test_split(df[['selftext']],df['googlehome'],test_size=0.25,\
                                                 stratify=df['googlehome'],\
                                                 random_state=42)

In [ ]:
# Equal proportion of classes split across train and test set
print(y_train.value_counts())
y_test.value_counts()

In [ ]:
y_test

In [ ]:
# Lines of text in test set and train set
lines_train = X_train.shape[0]
lines_test = X_test.shape[0]
print(f"Lines in train set: {lines_train}.")
print(f"Lines in test set: {lines_test}.")

In [ ]:
# Instantiate porterstemmer
p_stemmer = PorterStemmer()

In [ ]:
# Define function to convert a raw selftext to a string of words
# The input is a single string (a raw selftext), and 
# the output is a single string (a preprocessed selftext)

def selftext_to_words(raw_selftext):
    
    # 1. Remove HTML.
    selftext_text = BeautifulSoup(raw_selftext).get_text()
    
    # 2. Remove non-letters.
    letters_only = re.sub("[^a-zA-Z]", " ", selftext_text)
    
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    
    # 4. In Python, searching a set is much faster than searching
    # a list, so convert the stopwords to a set.
    stops = set(stopwords.words('english'))

    # 5. Remove stopwords.
    meaningful_words = [w for w in words if w not in stops]
    
    # 5.5 Stemming of words
    meaningful_words = [p_stemmer.stem(w) for w in words]
    
    # 6. Join the words back into one string separated by space, 
    # and return the result
    return(" ".join(meaningful_words))

In [ ]:
#Initialize an empty list to hold the clean reviews.
X_train_clean = []
X_test_clean = []

#For train set
# Instantiate counter.
j = 0
for text in X_train['selftext']:
    """Convert text to words, then append to cX_train_clean."""
    X_train_clean.append(selftext_to_words(text))
    
    # If the index is divisible by 1000, print a message.
    if (j + 1) % 1000 == 0:
        print(f'Clean & parse {j + 1} of {lines_train+lines_test}.')
    
    j += 1

# For test set
for text in X_test['selftext']:
    """Convert text to words, then append to cX_train_clean."""
    X_test_clean.append(selftext_to_words(text))
    
    # If the index is divisible by 1629, print a message.
    if (j + 1) % (lines_train+lines_test) == 0:
        print(f'Clean and parse {j + 1} of {lines_train+lines_test}.')
    
    j += 1

### Baseline accuracy
---
We first derive the baseline accuracy so as to be able to determine if the subsequent models are better than the baseline (null) model (predicting the plurality class).

In [ ]:
y_test.value_counts(normalize=True)

The Baseline accuracy is the percentage of the majority class. In this case, the baseline accuracy is 0.506143. 
This serves as benchmark for measuring model performance (i.e. model accuracy should be higher than this baseline).

### Logistic Regression Model
---

In [ ]:
# Set up a pipeline with two stages
# 1.CountVectorizer (transformer)
# 2.LogisticRegression (estimator)
pipe = Pipeline([('cvec',CountVectorizer()),\
                 ('logreg',LogisticRegression(solver='lbfgs',max_iter=200,random_state=42))\
                ])

In [ ]:
# Parameters of pipeline object
pipe.get_params()

In [ ]:
# Load pipeline object into GridSearchCV to tune CountVectorizer
# Search over the following values of hyperparameters:
# Maximum number of features fit (top no. frequent occur words): 2000, 3000, 4000, 5000
# Minimum number of documents (collection of text) needed to include token: 2, 3
# Maximum number of documents needed to include token: 90%, 95%
# Check (individual tokens) and also check (individual tokens and 2-grams).

# n-gram: 1 token, 1-gram, or 1 token, 2-gram.
pipe_params = {
    'cvec__max_features': [2_000,3_000,4_000,5_000],\
    'cvec__min_df': [2,3],\
    'cvec__max_df': [0.9,0.95],\
    'cvec__ngram_range': [(1, 1), (1,2)]\
}

In [ ]:
# Instantiate GridSearchCV.
"""pipe refers to the object to optimize."""
"""param_grid refer to parameter values to search."""
"""cv refers to number of cross-validate fold."""
gs = GridSearchCV(pipe,\
                  param_grid=pipe_params,\
                  cv=5)

In [ ]:
# Fit GridSearch to the cleaned training data.
gs.fit(X_train_clean,y_train)

In [ ]:
# Check the results of the grid search
# Note the Score is against one-fifth (hold-out set) of train data
print(f"Best parameters: {gs.best_params_}")
print(f"Best score: {gs.best_score_}")

In [ ]:
# Save best model as gs_model.
# Trained on four-fifth of data.
gs_model = gs.best_estimator_

In [ ]:
# Score model on training set & testing set
print(f"Accuracy on train set: {gs_model.score(X_train_clean, y_train)}")
print(f"Accuracy on test set: {gs_model.score(X_test_clean, y_test)}")

The model accuracy is higher than the baseline accuracy (0.506). However, the model is overfitted with 7% drop in test accuracy compared to train accuracy. 

In [ ]:
# Confusion matrix on the first log reg model
# Pass in true values, predicted values to confusion matrix
# Convert Confusion matrix into dataframe
# Positive class (class 1) is googlehome
preds = gs.predict(X_test_clean)
cm = confusion_matrix(y_test, preds)
cm_df = pd.DataFrame(cm,columns=['pred googlepixel','pred googlehome'], index=['Actual googlepixel','Actual googlehome'])
cm_df

The positive class (class 1) refers to `googlehome`. False positive means the observation is classified as `googlehome` when it is actually `googlepixel`. 
False negative means the ovservation is classified as `googlepixel` when it is actually `googlehome`.

In [ ]:
# return nparray as a 1-D array.
confusion_matrix(y_test, preds).ravel()
# Save TN/FP/FN/TP values.
tn, fp, fn, tp = confusion_matrix(y_test,preds).ravel()

In [ ]:
# Summary of metrics for first log reg model
spec = tn/(tn+fp)
sens = tp/(tp+fn)
print(f"Specificity: {round(spec,4)}")
print(f"Sensitivity: {round(sens,4)}")
print(f"Accuracy: {round(gs.score(X_test_clean,y_test),4)}")

The Receiver Operating Characteristic curve is a way to visualize the overlap between our positive class and negative class by moving our classification threshold from 0 to 1. The more area under the blue curve, the better separated the class distributions are.

In [ ]:
# To visualize the ROC AUC curve, first
# Create a dataframe called pred_df that contains:
# 1. The list of true values of our test set.
# 2. The list of predicted probabilities based on our model.

pred_proba = [i[1] for i in gs.predict_proba(X_test_clean)]

pred_df = pd.DataFrame({'true_values': y_test,
                        'pred_probs':pred_proba})
pred_df.head()

In [ ]:
# Import roc_auc_score.
from sklearn.metrics import roc_auc_score

In [ ]:
# Calculate ROC AUC.
roc_auc_score(pred_df['true_values'],pred_df['pred_probs'])

In [ ]:
#Create figure
plt.figure(figsize = (10,7))

# Create threshold values. (Dashed orange line in plot.)
thresholds = np.linspace(0, 1, 200)

# Define function to calculate sensitivity. (True positive rate.)
def TPR(df, true_col, pred_prob_col, threshold):
    true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
    false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
    return true_positive / (true_positive + false_negative)
    
# Define function to calculate 1 - specificity. (False positive rate.)
def FPR(df, true_col, pred_prob_col, threshold):
    true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
    false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
    return 1 - (true_negative / (true_negative + false_positive))
    
# Calculate sensitivity & 1-specificity for each threshold between 0 and 1.
tpr_values = [TPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]
fpr_values = [FPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]

# Plot ROC curve.
plt.plot(fpr_values, # False Positive Rate on X-axis
         tpr_values, # True Positive Rate on Y-axis
         label='ROC Curve')

# Plot baseline. (Perfect overlap between the two populations.)
plt.plot(np.linspace(0, 1, 200),
         np.linspace(0, 1, 200),
         label='baseline',
         linestyle='--')

# Label axes.
plt.title(f'ROC Curve with AUC = {round(roc_auc_score(pred_df["true_values"], pred_df["pred_probs"]),3)}', fontsize=22)
plt.ylabel('Sensitivity', fontsize=18)
plt.xlabel('1 - Specificity', fontsize=18)

# Create legend.
plt.legend(fontsize=16);

An ROC AUC of 1 means the positive and negative populations are perfectly separated and that the model is as good as it can get. The closer the ROC AUC is to 1, the better. (1 is the maximum score.)

What would be the impact on model accuracy with the removal of the discovered overlapping key words? Let's find out.

In [ ]:
# Set up a pipeline, pipe2 with two stages
# 1.CountVectorizer (transformer)
# 2.LogisticRegression (estimator)
# 3.Remove the words ['does', 'work', 'time', 'want', 'use', 'using', 'way', 'assistant', 've', 'tried'] 
# via CountVectorizer
pipe2 = Pipeline([('cvec',CountVectorizer(stop_words=['does', 'work', 'time', 'want', 'use',\
                                                      'using', 'way', 'assistant', 've', 'tried'])),\
                  ('logreg',LogisticRegression(solver='lbfgs',max_iter=200,random_state=42))\
                 ])

In [ ]:
# Parameters of pipeline object
pipe2.get_params()

In [ ]:
# Load pipeline object into GridSearchCV to tune CountVectorizer
# Search over the following values of hyperparameters:
# Maximum number of features fit (top no. frequent occur words): 2000, 3000, 4000, 5000
# Minimum number of documents (collection of text) needed to include token: 2, 3
# Maximum number of documents needed to include token: 90%, 95%
# Check (individual tokens) and also check (individual tokens and 2-grams).

# n-gram: 1 token, 1-gram, or 1 token, 2-gram.
pipe2_params = {
    'cvec__max_features': [2_000,3_000,4_000,5_000],\
    'cvec__min_df': [2,3],\
    'cvec__max_df': [0.9,0.95],\
    'cvec__ngram_range': [(1, 1), (1,2)]\
}

In [ ]:
# Instantiate GridSearchCV.
"""pipe refers to the object to optimize."""
"""param_grid refer to parameter values to search."""
"""cv refers to number of cross-validate fold."""
gs2 = GridSearchCV(pipe2,\
                  param_grid=pipe2_params,\
                  cv=5)

In [ ]:
# Fit GridSearch to the cleaned training data.
gs2.fit(X_train_clean,y_train)

In [ ]:
# Check the results of the grid search
# Note the Score is against one-fifth (hold-out set) of train data
print(f"Best parameters: {gs2.best_params_}")
print(f"Best score: {gs2.best_score_}")

In [ ]:
# Save best model as gs_model.
# Trained on four-fifth of data.
gs2_model = gs2.best_estimator_

In [ ]:
# Score model on training set & testing set
print(f"Accuracy on train set: {gs2_model.score(X_train_clean, y_train)}")
print(f"Accuracy on test set: {gs2_model.score(X_test_clean, y_test)}")

The removal of the identified key words has little impact on improving test accuracy score. It would appear the words removed in the second logistic regression model are not the ones that led to misclassifications. We could revisit this issue on the subsequent best performing model.

In [ ]:
# Confusion matrix on the second log reg model
# Pass in true values, predicted values to confusion matrix
# Convert Confusion matrix into dataframe
# Positive class (class 1) is googlehome
preds2 = gs2.predict(X_test_clean)
cm = confusion_matrix(y_test, preds2)
cm_df = pd.DataFrame(cm,columns=['pred googlepixel','pred googlehome'], index=['Actual googlepixel','Actual googlehome'])
cm_df

Not surprisingly, the number of False positive/ False negatives increased with the slight decreased in test accuracy, compared to the first log reg model. 

In [ ]:
# return nparray as a 1-D array.
confusion_matrix(y_test, preds2).ravel()
# Save TN/FP/FN/TP values.
tn, fp, fn, tp = confusion_matrix(y_test,preds2).ravel()

In [ ]:
# Summary of metrics for first log reg model
spec = tn/(tn+fp)
sens = tp/(tp+fn)
print(f"Specificity: {round(spec,4)}")
print(f"Sensitivity: {round(sens,4)}")
print(f"Accuracy: {round(gs2.score(X_test_clean,y_test),4)}")

In [ ]:
# To visualize the ROC AUC curve, first
# Create a dataframe called pred_df that contains:
# 1. The list of true values of our test set.
# 2. The list of predicted probabilities based on our model.

pred_proba = [i[1] for i in gs2.predict_proba(X_test_clean)]

pred_df = pd.DataFrame({'true_values': y_test,
                        'pred_probs':pred_proba})
pred_df.head()

In [ ]:
# Calculate ROC AUC.
roc_auc_score(pred_df['true_values'],pred_df['pred_probs'])

In [ ]:
#Create figure
plt.figure(figsize = (10,7))

# Create threshold values. (Dashed orange line in plot.)
thresholds = np.linspace(0, 1, 200)

# Define function to calculate sensitivity. (True positive rate.)
def TPR(df, true_col, pred_prob_col, threshold):
    true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
    false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
    return true_positive / (true_positive + false_negative)
    
# Define function to calculate 1 - specificity. (False positive rate.)
def FPR(df, true_col, pred_prob_col, threshold):
    true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
    false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
    return 1 - (true_negative / (true_negative + false_positive))
    
# Calculate sensitivity & 1-specificity for each threshold between 0 and 1.
tpr_values = [TPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]
fpr_values = [FPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]

# Plot ROC curve.
plt.plot(fpr_values, # False Positive Rate on X-axis
         tpr_values, # True Positive Rate on Y-axis
         label='ROC Curve')

# Plot baseline. (Perfect overlap between the two populations.)
plt.plot(np.linspace(0, 1, 200),
         np.linspace(0, 1, 200),
         label='baseline',
         linestyle='--')

# Label axes.
plt.title(f'ROC Curve with AUC = {round(roc_auc_score(pred_df["true_values"], pred_df["pred_probs"]),3)}', fontsize=22)
plt.ylabel('Sensitivity', fontsize=18)
plt.xlabel('1 - Specificity', fontsize=18)

# Create legend.
plt.legend(fontsize=16);

We see that effect in removal of the identified common overlapping words is minimal on roc_auc is minimal (i.e. roc_auc of first and second logistic regression models).

### Naive Bayes Model
---

In [ ]:
# Set up a pipeline, p3 with two stages
# 1.CountVectorizer (transformer)
# 2.Naive Bayes(multinomial) (estimator)
pipe3 = Pipeline([('cvec',CountVectorizer()),\
                 ('nb',MultinomialNB())\
                ])

In [ ]:
# Parameters of pipeline object
pipe3.get_params()

In [ ]:
# Load pipeline object into GridSearchCV to tune CountVectorizer
# Search over the following values of hyperparameters:
# Maximum number of features fit (top no. frequent occur words): 2000, 3000, 4000, 5000
# Minimum number of documents (collection of text) needed to include token: 2, 3
# Maximum number of documents needed to include token: 90%, 95%
# Check (individual tokens) and also check (individual tokens and 2-grams).

# n-gram: 1 token, 1-gram, or 1 token, 2-gram.
pipe_params = {
    'cvec__max_features': [2_000,3_000,4_000,5_000],\
    'cvec__min_df': [2,3],\
    'cvec__max_df': [0.9,0.95],\
    'cvec__ngram_range': [(1, 1), (1,2)]\
}

In [ ]:
# Instantiate GridSearchCV.
"""pipe refers to the object to optimize."""
"""param_grid refer to parameter values to search."""
"""cv refers to number of cross-validate fold."""
gs3 = GridSearchCV(pipe3,\
                  param_grid=pipe_params,\
                  cv=5)

In [ ]:
# Fit GridSearch to the cleaned training data.
gs3.fit(X_train_clean,y_train)

In [ ]:
# Check the results of the grid search
# Note the Score is against one-fifth (hold-out set) of train data
print(f"Best parameters: {gs3.best_params_}")
print(f"Best score: {gs3.best_score_}")

In [ ]:
# Save best model as gs_model.
# Trained on four-fifth of data.
gs_model = gs3.best_estimator_

In [ ]:
# Score model on training set & testing set
print(f"Accuracy on train set: {gs_model.score(X_train_clean, y_train)}")
print(f"Accuracy on test set: {gs_model.score(X_test_clean, y_test)}")

The naive bayes test accuracy is higher than the train accuracy, so the model doesn't appear to be overfitted on the training data.
Compared to the logistic regression models, naive bayes model has:
- lower accuracy on train set
- higher accuracy on test set.

The naive bayes model performs better than the logistic regression model.

In [ ]:
# Confusion matrix on the first naive bayes model
# Pass in true values, predicted values to confusion matrix
# Convert Confusion matrix into dataframe
# Positive class (class 1) is googlehome
preds3 = gs3.predict(X_test_clean)
cm = confusion_matrix(y_test, preds3)
cm_df = pd.DataFrame(cm,columns=['pred googlepixel','pred googlehome'], index=['Actual googlepixel','Actual googlehome'])
cm_df

With a higher test accuracy, there is a lower number of False positive/ False negatives in test prediction. Almost equal number of false positives and false negatives.

In [ ]:
# return nparray as a 1-D array.
confusion_matrix(y_test, preds3).ravel()
# Save TN/FP/FN/TP values.
tn, fp, fn, tp = confusion_matrix(y_test,preds3).ravel()

In [ ]:
# Summary of metrics for first naive bayes model
spec = tn/(tn+fp)
sens = tp/(tp+fn)
print(f"Specificity: {round(spec,4)}")
print(f"Sensitivity: {round(sens,4)}")
print(f"Accuracy: {round(gs3.score(X_test_clean,y_test),4)}")

In [ ]:
# To visualize the ROC AUC curve, first
# Create a dataframe called pred_df that contains:
# 1. The list of true values of our test set.
# 2. The list of predicted probabilities based on our model.

pred_proba = [i[1] for i in gs3.predict_proba(X_test_clean)]

pred_df = pd.DataFrame({'true_values': y_test,
                        'pred_probs':pred_proba})
pred_df.head()

In [ ]:
# Calculate ROC AUC.
roc_auc_score(pred_df['true_values'],pred_df['pred_probs'])

In [ ]:
#Create figure
plt.figure(figsize = (10,7))

# Create threshold values. (Dashed blue line in plot.)
thresholds = np.linspace(0, 1, 200)

# Define function to calculate sensitivity. (True positive rate.)
def TPR(df, true_col, pred_prob_col, threshold):
    true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
    false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
    return true_positive / (true_positive + false_negative)
    
# Define function to calculate 1 - specificity. (False positive rate.)
def FPR(df, true_col, pred_prob_col, threshold):
    true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
    false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
    return 1 - (true_negative / (true_negative + false_positive))
    
# Calculate sensitivity & 1-specificity for each threshold between 0 and 1.
tpr_values = [TPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]
fpr_values = [FPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]

# Plot ROC curve.
plt.plot(fpr_values, # False Positive Rate on X-axis
         tpr_values, # True Positive Rate on Y-axis
         label='ROC Curve')

# Plot baseline. (Perfect overlap between the two populations.)
plt.plot(np.linspace(0, 1, 200),
         np.linspace(0, 1, 200),
         label='baseline',
         linestyle='--')

# Label axes.
plt.title(f'ROC Curve with AUC = {round(roc_auc_score(pred_df["true_values"], pred_df["pred_probs"]),3)}', fontsize=22)
plt.ylabel('Sensitivity', fontsize=18)
plt.xlabel('1 - Specificity', fontsize=18)

# Create legend.
plt.legend(fontsize=16);

## ? Check w TA how does one go about describing the ROC curve??

### Model Performance Summary and Review of Production Model
---

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Define function to print model performance metrics and roc auc
def scores(model):
    print(classification_report(y_test, model.predict(X_test_clean)))
    pred_proba = [i[1] for i in model.predict_proba(X_test_clean)]
    print(f"roc_auc: {round(roc_auc_score(pred_df['true_values'],pred_df['pred_probs']),3)}")

In [ ]:
# Summary scores for Logistic Regression Model
scores(gs)

In [ ]:
# Summary scores for Logistic Regression Model (words removed)
# List of words removed
# ['does' 'work', 'time', 'want', 'use', 'using', 'way', 'assistant', 've', 'tried']
scores(gs2)

In [ ]:
# Summary scores for Naive Bayes model
scores(gs3)

Deeper Look at Classifications of the Naive Bayes Model 
---
For the test accuracy and roc_auc scores, the Naive Bayes is selected as the better performing model. This section examines the features that
- helps with negative (googlepixel) and positive (googlehome) classifications,
- what could be the features that lead to misclassifications.

In [ ]:
# Review model coefficients to see which word is helping with negative / positive classifications)
# As GridSearchCV' object has no attribute 'feature_log_prob_,
# Build separate naive bayes model to enable model coefficient extraction
# using best parameters discovered above in gs3

# Instantiate our CountVectorizer
cv = CountVectorizer(ngram_range=(1,2),max_df=0.9,min_df=3,max_features=3000)

# Fit and transform training data
X_train_cleancv = cv.fit_transform(X_train_clean)

# Transform test data
X_test_cleancv = cv.transform(X_test_clean)

In [ ]:
# Instantiate model
nb = MultinomialNB()

# Fit model
model = nb.fit(X_train_cleancv,y_train)

# Generate predictions
predictions = nb.predict(X_test_cleancv)

In [ ]:
import numpy as np
#prob for positive class
pos_class_prob_sorted = nb.feature_log_prob_[1, :].argsort()
#prob for negative class
neg_class_prob_sorted = nb.feature_log_prob_[0, :].argsort()
#getting the top features 
pos_top_features = np.take(cvec.get_feature_names(), pos_class_prob_sorted)
neg_top_features = np.take(cvec.get_feature_names(), neg_class_prob_sorted)


In [ ]:
# List of top 30 words that helps in 'googlehome' (positive) classification
print(pos_top_features[:30])

In [ ]:
# List of top 30 words that helps in 'googlepixel' (negative) classification
print(neg_top_features[:30])

Surprisingly, the top 20 words that aided in postive/ negative classifications are neither `google home` nor `google pixel`. 

**Googlehome**: For positive class (googlehome), it could be inferred that the users reviews seems quite balanced. The word `apple` surfaced, possibly as a mention as comparison of `apple` product performance-related topics with google's. Further analysis could be done into contents with word of negative conotation such as `frustrating`, `frustrated`, `fucking` to better understand the technical issues causing users' frustrations. The main issues with IoT devices are interoperability and connectivity, but it remains to be seen if this is the case for the current google home reddit text analysis.

**Googlepixel**: For negative class (googlepixel), there are more words (eight) with negative conotations, from `horrible battery` to `dumb` and `bad batch`. Typical smartphone issues range from dismal battery life to poor build quality and these words seem to reinforce this view. The google R&D and production lines would do well to learn from these negative user reviews.

**Reviewing the misclassified samples**: What could be the features that lead to misclassifications?

In [ ]:
# Pass y_test (pandas series) into dataframe first
# in order to use the original selftext indexes for traceability
actual = pd.Series(y_test)
df_rvw = actual.to_frame()

# Create column of predicted classes from Naive Bayes model
df_rvw['pred'] = preds3
# Include the selftext data
df_rvw['selftext'] = X_test_clean

# Review the dataframe
df_rvw.head()

In [ ]:
# Index of misclassified classes
row_ids = df_rvw[df_rvw['googlehome'] != df_rvw['pred']].index
row_ids

In [ ]:
# Create overview of the misclassified 
for i in row_ids:
    print(df_rvw.loc[i])

Two samples from each of misclassified samples are examined in further detail in this section.

**False Negative**: Predicted as googlepixel post but is actually googlehome post. Potential reason could be due to the presence of words such as `phone` and `updat` that are more common in the context of smartphones.

In [ ]:
# Example 1 of false negative classification
print(df_rvw['selftext'][603])

In [ ]:
# Example 2 of false negative classification
print(df_rvw['selftext'][525])

**False Positive**: Predicted as googlehome post but is actually googlepixel post. Potential reason could be due to the presence of words that are related to phone and phone calls (google home can make home calls) that lead to misclassification as pixel (smartphone) post.

In [ ]:
# Example 1 of false positive classification
print(df_rvw['selftext'][1498])

In [ ]:
# Example 2 of false positive classification
print(df_rvw['selftext'][1024])

### Recommendations and Way Forward

Model good to go. Further enhancements could be implemented in version 2. What more can be done is to switch to review rival product posts for more insights.

---